<h1 style="font-size: 22px; line-height: 100%; text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142); border-radius: 10px;">
External Sub-network Search with BootstrapNAS and DyNAS-T
</h1>

This notebook demonstrates how to use an external search solution to find high-performing subnetworks in NNCF's [BootstrapNAS](https://github.com/openvinotoolkit/nncf/tree/develop/examples/experimental/torch/classification). We will use an existing super-network generated from ResNet-50.  

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Imports and Settings
</h3>

In [1]:
import sys
import os
sys.path.append('..')
sys.path.append('/localdisk/maciej/code/dynast_bootstrapnas_integration/nncf/')

from imports_bnas import * # Import NNCF, PyTorch and other required packages.
import os
from examples.common.sample_config import SampleConfig
from nncf.common.initialization.batchnorm_adaptation import BatchnormAdaptationAlgorithm
from nncf.experimental.torch.nas.bootstrapNAS.search.supernet import TrainedSuperNet
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.multi_elasticity_handler import SubnetConfig
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.elasticity_dim import ElasticityDim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

MODEL_DIR = Path("../../../models/supernets/cifar10/resnet50")
OUTPUT_DIR = Path("output")
DATA_DIR = Path("data")
BASE_MODEL_NAME = "resnet-50"

fp32_pth_path, model_onnx_path, supernet_onnx_path, subnet_onnx_path = create_folders_demo(BASE_MODEL_NAME + '_external_search')


Imported PyTorch and NNCF
INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch
Using cuda device


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Loading an existing super-network
</h3>

In [2]:
model = resnet50_cifar10()
config = SampleConfig.from_json(os.path.join(MODEL_DIR, 'config.json'))
config.device = device
print(MODEL_DIR)
supernet = TrainedSuperNet.from_checkpoint(model, config, os.path.join(MODEL_DIR, 'elasticity.pth'), os.path.join(MODEL_DIR, 'supernet_weights.pth'))

../../../models/supernets/cifar10/resnet50


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Explore the Search Space
</h3>



In [3]:
supernet.get_search_space()

{'width': {0: [256, 200, 152, 128],
  1: [512, 408, 304, 256],
  2: [1024, 816, 608, 512],
  3: [2048, 1632, 1224, 1024],
  4: [64, 48, 32],
  5: [64, 48, 32],
  6: [64, 48, 32],
  7: [64, 48, 32],
  8: [64, 48, 32],
  9: [64, 48, 32],
  10: [64, 48, 32],
  11: [128, 96, 72, 64],
  12: [128, 96, 72, 64],
  13: [128, 96, 72, 64],
  14: [128, 96, 72, 64],
  15: [128, 96, 72, 64],
  16: [128, 96, 72, 64],
  17: [128, 96, 72, 64],
  18: [128, 96, 72, 64],
  19: [256, 200, 152, 128],
  20: [256, 200, 152, 128],
  21: [256, 200, 152, 128],
  22: [256, 200, 152, 128],
  23: [256, 200, 152, 128],
  24: [256, 200, 152, 128],
  25: [256, 200, 152, 128],
  26: [256, 200, 152, 128],
  27: [256, 200, 152, 128],
  28: [256, 200, 152, 128],
  29: [256, 200, 152, 128],
  30: [256, 200, 152, 128],
  31: [512, 408, 304, 256],
  32: [512, 408, 304, 256],
  33: [512, 408, 304, 256],
  34: [512, 408, 304, 256],
  35: [512, 408, 304, 256],
  36: [512, 408, 304, 256]},
 'depth': [[0],
  [2],
  [4],
  [5],
  

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Data Loaders for validation and batchnorm adaptation
</h3>

Note: Number of samples used for Batch Norm Adaptation might greatly influence the top1 evaluation for each subnetwork.
`6000` is a good value for ResNet-50 CIFAR10, but it is advised to perform an ablation study for each new architecture to identify the best number of samples.

In [4]:
DATASET_DIR = DATA_DIR / "cifar10"

batch_size_val = 1000
batch_size = 64
train_loader, val_loader = create_cifar10_dataloader(DATASET_DIR, batch_size, batch_size_val, device)

bn_adapt_args = BNAdaptationInitArgs(data_loader=wrap_dataloader_for_init(train_loader), device=config.device)

bn_adapt_algo_kwargs = {
    'data_loader': train_loader,
    'num_bn_adaptation_samples': 6000,
    'device': 'cuda',
}
bn_adaptation = BatchnormAdaptationAlgorithm(**bn_adapt_algo_kwargs)

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Sample sub-networks to implement your own algorithm
</h3>


In [5]:
# Implement your function to validate 

def acc_top1(model):
    from bootstrapnas_utils import validate

    bn_adaptation.run(model)

    model.eval()
    model.to(device)
    model_top1_acc, _, _ = validate(model, val_loader) 
    return model_top1_acc

supernet.activate_maximal_subnet()
supernet.eval_active_subnet(acc_top1)

Test: [ 0/10]	Time 0.680 (0.680)	Loss 0.314 (0.314)	Acc@1 93.30 (93.30)	Acc@5 99.00 (99.00)
 * Acc@1 93.660 Acc@5 98.990


tensor(93.6600, device='cuda:0')

In [6]:
subnet_config = SubnetConfig()
subnet_config[ElasticityDim.DEPTH] = [1]

supernet.eval_subnet(subnet_config, acc_top1)

Test: [ 0/10]	Time 0.567 (0.567)	Loss 0.313 (0.313)	Acc@1 93.40 (93.40)	Acc@5 98.90 (98.90)
 * Acc@1 93.670 Acc@5 99.030


tensor(93.6700, device='cuda:0')

In [7]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

325.799936

In [8]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

325.799936

In [9]:
subnet_config[ElasticityDim.DEPTH] = [0]
subnet_config[ElasticityDim.WIDTH] = {2: 512}

supernet.activate_config(subnet_config)
supernet.get_macs_for_active_config()

272.32256

In [10]:
supernet.get_active_config()

OrderedDict([(<ElasticityDim.WIDTH: 'width'>,
              {0: 256,
               1: 512,
               2: 512,
               3: 2048,
               4: 64,
               5: 64,
               6: 64,
               7: 64,
               8: 64,
               9: 64,
               10: 64,
               11: 128,
               12: 128,
               13: 128,
               14: 128,
               15: 128,
               16: 128,
               17: 128,
               18: 128,
               19: 256,
               20: 256,
               21: 256,
               22: 256,
               23: 256,
               24: 256,
               25: 256,
               26: 256,
               27: 256,
               28: 256,
               29: 256,
               30: 256,
               31: 512,
               32: 512,
               33: 512,
               34: 512,
               35: 512,
               36: 512}),
             (<ElasticityDim.DEPTH: 'depth'>, [0])])

In [11]:
supernet.eval_subnet(subnet_config, acc_top1)

Test: [ 0/10]	Time 0.562 (0.562)	Loss 0.329 (0.329)	Acc@1 92.70 (92.70)	Acc@5 98.80 (98.80)
 * Acc@1 93.740 Acc@5 98.950


tensor(93.7400, device='cuda:0')

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Search using DyNAS-t
</h3>

![DyNAS-T Logo](https://github.com/IntelLabs/DyNAS-T/blob/main/docs/images/dynast_logo.png?raw=true)

DyNAS-T (Dynamic Neural Architecture Search Toolkit) is a super-network neural architecture search NAS optimization package designed for efficiently discovering optimal deep neural network (DNN) architectures for a variety of performance objectives such as accuracy, latency, multiply-and-accumulates, and model size.

DyNAS-T is compatible with supernetworks created with the NNCF BootstrapNAS. The search on NNCF BootstrapNAS supernetworks can be performed in just few steps, as shown below.

In [18]:
%load_ext autoreload
%autoreload 2

from dynast.dynast_manager import DyNAS

dynast_config = {
    'results_path': 'bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv',
    'search_tactic': 'linas',
    'num_evals': 250,
    'population': 50,
    'seed': 42,
    'supernet': 'bootstrapnas_image_classification',  # this parameter works in a tangent with `bootstrapnas_supernetwork` and should be set to `bootstrapnas_image_classification` for all CNN BootstrapNAS supernetworks;
    'batch_size': config.batch_size,
    'bootstrapnas_supernetwork': supernet,  # pass the instance of nncf.experimental.torch.nas.bootstrapNAS.search.supernet.TrainedSuperNet to the search algorithm;
    'device': config.device,
    'verbose': False,
    'optimization_metrics': ['accuracy_top1', 'macs'],  # define optimization metrics that will guide the search progression;
    'measurements': ['accuracy_top1', 'macs'],  # define measurements that will be collected during the search; This is a superset of optimization metrics;
    'metric_eval_fns': {
        'accuracy_top1': acc_top1,  # register custom evaluation function;
    }
}

dynas = DyNAS(**dynast_config)

[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:51 - ========================================
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:52 - Starting Dynamic NAS Toolkit (DyNAS-T)
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:53 - ========================================
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:77 - ----------------------------------------
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:78 - DyNAS Parameter Inputs:
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:80 - results_path: bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:80 - num_evals: 1
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:80 - population: 1
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:80 - seed: 42
[07-05 11:01:23 MainProcess #63579] INFO  dynast_manager.py:80 - supernet: bootstrapnas_image_classification
[07-05 11:01:23 Main

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:

dynas.search()

[07-05 11:02:41 MainProcess #63579] INFO  evaluation_interface.py:49 - (Re)Formatted results file: bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv
[07-05 11:02:41 MainProcess #63579] INFO  evaluation_interface.py:50 - csv file header: ['Sub-network', 'Date', 'Model Parameters', 'Latency (ms)', 'MACs', 'Top-1 Acc (%)']
[07-05 11:02:41 MainProcess #63579] INFO  search_tactic.py:728 - Evaluating subnetwork 1/1


[07-05 11:02:42 MainProcess #63579] INFO  bootstrapnas_interface.py:84 - Using custom accuracy_top1 metric evaluation function.


Test: [ 0/10]	Time 0.550 (0.550)	Loss 0.329 (0.329)	Acc@1 93.20 (93.20)	Acc@5 99.20 (99.20)


[07-05 11:02:50 MainProcess #63579] INFO  bootstrapnas_interface.py:108 - Validation accuracy: 93.72%


 * Acc@1 93.720 Acc@5 99.360


[07-05 11:02:51 MainProcess #63579] INFO  __init__.py:76 - > Calling get_macs
[07-05 11:02:52 MainProcess #63579] INFO  __init__.py:81 - > Finished get_macs in 1.1748 s


[OrderedDict([(<ElasticityDim.WIDTH: 'width'>,
               {0: 152,
                1: 512,
                2: 512,
                3: 2048,
                4: 48,
                5: 64,
                6: 32,
                7: 48,
                8: 32,
                9: 32,
                10: 48,
                11: 96,
                12: 128,
                13: 128,
                14: 96,
                15: 72,
                16: 128,
                17: 96,
                18: 128,
                19: 128,
                20: 152,
                21: 128,
                22: 152,
                23: 200,
                24: 152,
                25: 152,
                26: 200,
                27: 152,
                28: 256,
                29: 200,
                30: 200,
                31: 408,
                32: 256,
                33: 256,
                34: 304,
                35: 408,
                36: 304}),
              (<ElasticityDim.DEPTH: 'depth'>,